In [1]:
train_path = '/content/drive/MyDrive/DataColab/NLP_project/Smart Tech CV Parser & Matcher/DataSets/final DataSet/trainLabelEncoder.csv'
test_paht = '/content/drive/MyDrive/DataColab/NLP_project/Smart Tech CV Parser & Matcher/DataSets/final DataSet/testLabelEncoder.csv'

# **Environment Setup**

In [ ]:
#!pip install transformers datasets evaluate scikit-learn

# **Loading the Dataset & Train/Test**

In [2]:
from datasets import load_dataset

datasets = load_dataset('csv' , data_files= train_path)
datasets = datasets['train'].train_test_split(test_size = 0.2)

In [3]:
datasets = datasets.rename_columns({'Category' : 'labels'})

# **Tokenizer**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(text):
    return tokenizer(
    text["Text"],
    padding="max_length",
    truncation=True
)
datasets = datasets.map(tokenize_function, batched=True)

# **Loading the BERT Model**

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=23)

# **Training Configuration**

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    )

# **Metrics Computation**

In [ ]:
!pip install evaluate

In [8]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# **Creating the Trainer**

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    compute_metrics=compute_metrics,
    )

In [ ]:
trainer.train()

# **Training & Evaluation**

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate()

#**Save**

In [ ]:
trainer.save_model("./my_model")
tokenizer.save_pretrained("./my_model")

# **Inference Pipeline**

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./my_model")
print(classifier( 'CV' ))